# Post-correction
---

#### Overview
Prior to import to render-ws, apply post-correction to FAST-EM images. This notebook supports interactive selection of the optimal parameters for post-correction.  
Optimal parameters can be applied to all data via the command line script or in the notebook.

In [2]:
import pathlib
from natsort import natsorted
import fnmatch
import logging

from scripted_render_pipeline.postcorrector.post_corrector import Post_Corrector
from interactive_render.plotting import plot_images_with_artefacts

# indirectly enable autocomplete
%config Completer.use_jedi = False

# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Parameters

* `dir_project` : path to project directory -- assumes subdirectories are the ROA's (which are further assumed to be sections)
* `project_paths (optional)` : paths to ROA directories -- assumes contents are (a grid of) tiff files
* `parallel`: how many threads to use in parallel, optimises io usage
* `clobber`: whether to allow overwriting of corrected images
* `pct` : Percentile used in artifact detection
*  `a` : Scaling factor for thresholding the deviation from the median. Increasing `a` will allow for larger deviations

In [3]:
# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/")

# set parameters for correction
params_process = {
    "parallel": 40,
    "clobber": True,
    "pct": 0.1,
    "a": 1
}

# collect filepaths to ROA's
fps_ROA = natsorted([fp for fp in dir_project.glob("*") if (fp.is_dir() and fnmatch.fnmatch(fp.name, 'S*'))])
fps_ROA = {k: v for k, v in enumerate(fps_ROA)}
fps_ROA

{0: PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S000'),
 1: PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S001'),
 2: PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S002')}

#### Inspect section

In [4]:
z = 1
filepaths_raw = [fp for fp in fps_ROA[z].glob("*.tiff")]
filepaths_raw[0:5]

[PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S001/000_000_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S001/000_001_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S001/000_002_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S001/000_003_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S001/000_004_0.tiff')]

In [5]:
# Calculate median and median absolute deviation (mad)
pct = 0.1
postcorrector = Post_Corrector(project_path=dir_project)
med = postcorrector.get_med(filepaths=filepaths_raw, 
                            pct=params_process["pct"])
mad = postcorrector.get_mad(filepaths=filepaths_raw, 
                            med=med, 
                            pct=params_process["pct"])

In [6]:
# Visualize fields with artefacts
plot_images_with_artefacts(filepaths=filepaths_raw,
                           med=med,
                           mad=mad)

  0%|          | 0/25 [00:00<?, ?it/s]

interactive(children=(FloatText(value=0.1, description='percentile'), FloatText(value=1.0, description='scalin…

#### Get MED and MAD from sample of images in all sections

In [8]:
import random

In [9]:
n = 10
fps_total = []
for z in fps_ROA:
    fps_sample = random.sample([fp for fp in fps_ROA[z].glob("*.tiff")], n)
    fps_total = fps_total + fps_sample

In [10]:
# Calculate median and median absolute deviation (mad) from sample
postcorrector = Post_Corrector(project_path=dir_project)
med = postcorrector.get_med(filepaths=fps_total, 
                            pct=params_process["pct"])
mad = postcorrector.get_mad(filepaths=fps_total, 
                            med=med, 
                            pct=params_process["pct"])

Inspect section

In [15]:
z = 0
filepaths_raw = [fp for fp in fps_ROA[z].glob("*.tiff")]
filepaths_raw[0:5]

[PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S000/000_000_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S000/000_001_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S000/000_002_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S000/000_003_0.tiff'),
 PosixPath('/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/S000/000_004_0.tiff')]

In [16]:
# Visualize fields with artefacts
plot_images_with_artefacts(filepaths=filepaths_raw,
                           med=med,
                           mad=mad,
                           )

  0%|          | 0/25 [00:00<?, ?it/s]

interactive(children=(FloatText(value=0.1, description='percentile'), FloatText(value=1.0, description='scalin…

#### Post-correct all sections

In [7]:
project_paths = [p for p in dir_project.iterdir() if p.is_dir() and fnmatch.fnmatch(p.name, 'S*')]
project_paths

[PosixPath('/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S056'),
 PosixPath('/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S054'),
 PosixPath('/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S055')]

In [9]:
postcorrector = Post_Corrector(project_path=dir_project,
                               parallel=params_process["parallel"],
                               clobber=params_process["clobber"],
                               pct=0.1, # Set to optimal value
                               a=3, # Set to optimal value
                               project_paths=project_paths
                               )

failed_sections = postcorrector.post_correct_all_sections()
if not failed_sections:
    logging.info("post-correction completed succesfully")
else:
    logging.info("Post_correction failed for: %s", [section.name for section in failed_sections])      
    logging.info("Detected failed sections. Rerunning post-correction using nearest available correction image")
    postcorrector.post_correct_failed_sections(failed_sections)

post-correcting sections: 100%|██████████| 3/3 [05:42<00:00, 114.05s/sections]
